In [1]:
import time
import string
from datetime import datetime

## For vector database
import lancedb

## For data handling
import polars as pl

## For embedding model
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
CONFIG = "00"
VERBOSE = True

## Embedding model
MODEL = HuggingFaceEmbeddings(model_name="thenlper/gte-base")

## ANN query parameters
## https://lancedb.github.io/lancedb/concepts/index_ivfpq/#query-the-index
LIMIT=20
NPROBES=20
REFINE_FACTOR=10

DB_DIR = "../db_data/db_"+CONFIG
TABLE = "table_"+CONFIG

## Query results
# Get current date and time
now = datetime.now()
# Format the timestamp (YYYYMMDDHHMM)
timestamp = now.strftime("%Y%m%d%H%M")
# Standard name with timestamp
filename = f"labeled_data_{timestamp}.parquet"

QUERY_RATINGS_PATH = "../labeled_data/"+filename

In [3]:
## Connect to the database we've already created
db = lancedb.connect(DB_DIR)

## Load the table we've already created
table = db.open_table(TABLE)

def ask_a_query(query_text, print_results=False):
    ## Start the clock
    start_time = time.time()
    
    ## Embed the query
    query = MODEL.embed_query(query_text)

    ## Perform similarity search on database using ANN
    result = table.search(query) \
        .limit(LIMIT) \
        .nprobes(NPROBES) \
        .refine_factor(REFINE_FACTOR) \
        .to_polars()

    ## Stop the clock
    end_time = time.time()
    print(f"Results retrieved in {end_time-start_time:.4f} seconds.\n")

    if print_results:
        ## Print the basics components of the result...
        pl.Config.set_tbl_rows(25)
        pl.Config.set_fmt_str_lengths(3000)
        print(result[["aware_post_type","reddit_subreddit", "reddit_text", "text_chunk", "_distance"]])
        pl.Config.set_tbl_rows(10)
        pl.Config.set_fmt_str_lengths(20)

    return result

In [4]:
result = ask_a_query("What is your favorite thing about working for Startbucks?")

Results retrieved in 0.2371 seconds.



In [5]:
pl.Config.set_tbl_rows(25)
pl.Config.set_fmt_str_lengths(3000)
result[["reddit_name", 
        "reddit_subreddit", 
        "text_chunk",
        "reddit_text", 
        "_distance"]]

reddit_name,reddit_subreddit,text_chunk,reddit_text,_distance
str,str,str,str,f32
"""t3_1b2birc""","""Lowes""","""What’s your favorite part about working here?""","""What’s your favorite part about working here?""",0.22089
"""t1_kijhj7c""","""TalesFromYourBank""","""I overall like my job... it has more pros for sure then Cons. My favorite things: • It's awesome when you have a customer who remembers you helping them - I had one recently come I the location I now manage and mention that they remembered me when I just started out helping them rebuild their credit after a messy divorce. That was a good feeling, to know I made a genuine impact with my advice and resources.""","""I was always told growing up there is no perfect job. There's always going to be a part you don't like - or parts. I overall like my job... it has more pros for sure then Cons. My favorite things: • It's awesome when you have a customer who remembers you helping them - I had one recently come I the location I now manage and mention that they remembered me when I just started out helping them rebuild their credit after a messy divorce. That was a good feeling, to know I made a genuine impact with my advice and resources. • I've also been able to help relieve financial burdens of clients --- lowering payments from $2500-$4000/month down to half of that amount or less. That's an awesome feeling. • Seeing my employees catch on to something or be successful is the best feeling overall. Yeah, a huge chunk of my job now is managing sales and production BUT my personal agenda is seeing success through making my people successful. If they win, the business wins. • Having Federal Holidays off is nice, too. Disadvantages: • Dealing with some ... interesting people. At my particular location, I'm in the rougher part of town and people don't think twice about cussing, yelling, and making a ruckus. That was and still is at time a shock to me because I never worked in a branch with tolerance to that kind of behavior. I am naturally someone who doesn't like to get involved with conflict, too. And while it's listed as a negative for me, I have had to get stern with people who were regular rough-necks and tell them I want to help them but can't with cussing and yelling - you'd be surprised how many of them straighten up and now they come to me for everything (new grandbaby birth announcements, loan requests, referring business). Thus also has helped me build my confidence in handling difficult situations and conversations. • Robberies. 'Nuff said. • Being a manager is often times a thankless job... it'd be nice for someone besides my own boss to give me a high five for hitting my own individual goals that I am required to carry for the branch. But, again, not a huge set back because I understand I signed up for that by swapping positions.""",0.24502
"""t1_iw5rynn""","""TalesFromYourBank""","""I love that my job is flexible on start and end time, reliable, has excellent benefits that help make up for the poor salary. I'm in the back office and I don't have to deal with customers 99% of the time. Fraud detection and prevention is my favorite part!""","""I love that my job is flexible on start and end time, reliable, has excellent benefits that help make up for the poor salary. I'm in the back office and I don't have to deal with customers 99% of the time. Fraud detection and prevention is my favorite part!""",0.250641
"""t1_jwxd9xu""","""starbucks""","""My fav thing is asking customers what they do for a living (:""","""My fav thing is asking customers what they do for a living (:""",0.259571
"""t3_1anikwu""","""starbucks""","""the fact that I’m able to pay rent. This job is hard enough with Starbuck’s high standards for us. Please just try to be considerate.""","""Okay, I wrote this in April of 2023, and I’ve actually quit Starbucks since then. I spent over a year there and honestly miss the job. A lot of customers are really nice and just trying to get a coffee. However, si

In [7]:
pl.Config.set_tbl_rows(10)
pl.Config.set_fmt_str_lengths(30)

polars.config.Config